In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
!pip2 install git+https://github.com/stanbiryukov/sklearn-GLMM

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Cloning https://github.com/stanbiryukov/sklearn-GLMM to /tmp/pip-req-build-LVxt0Q
  Running command git clone -q https://github.com/stanbiryukov/sklearn-GLMM /tmp/pip-req-build-LVxt0Q
  Created wheel for sklearn-GLMM: filename=sklearn_GLMM-0.1.1-cp27-none-any.whl size=6016 sha256=8bc962f56f09fc2b6695c1351e56b2284a9178d5cb2bf68914dea36f44f7d1f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-HCzdQK/wheels/f0/62/46/8c288791f884c5f11b79be91b8b3d40833c5e40b04cb9b20db
Successfully built sklearn-GLMM
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from pyGLMM import skGLMM, r_formula
# example from https://stats.idre.ucla.edu/r/dae/mixed-effects-logistic-regression/

In [4]:
df = pd.read_csv('https://stats.idre.ucla.edu/stat/data/hdp.csv')
# Factorize non-numeric
df = df.apply(lambda x: pd.factorize(x)[0] if np.issubdtype(x.dtype, np.number) is False else x)
df.head()

,tumorsize,co2,pain,wound,mobility,ntumors,nmorphine,remission,lungcapacity,Age,Married,FamilyHx,SmokingHx,Sex,CancerStage,LengthofStay,WBC,RBC,BMI,IL6,CRP,DID,Experience,School,Lawsuits,HID,Medicaid
0,67.981196,1.534333,4,4,2,0,0,0,0.801088,64.968239,0,0,0,0,0,6,6087.648615,4.868416,24.144237,3.698981,8.086417,1,25,0,3,1,0.605867
1,64.702458,1.676132,2,3,2,0,0,0,0.326444,53.917136,0,0,0,1,0,6,6700.310486,4.679052,29.405162,2.627481,0.803488,1,25,0,3,1,0.605867
2,51.567001,1.533445,6,3,2,0,0,0,0.565031,53.347302,1,0,1,1,0,5,6042.809114,5.005862,29.482586,13.896153,4.034157,1,25,0,3,1,0.605867
3,86.437990,1.453300,3,3,2,0,0,0,0.848411,41.368043,0,0,0,0,1,5,7162.696543,5.265058,21.557259,3.008033,2.125863,1,25,0,3,1,0.605867
4,53.400177,1.566348,3,4,2,0,0,0,0.886491,46.800422,0,0,1,0,0,6,6443.440465,4.984259,29.815188,3.890698,1.349324,1,25,0,3,1,0.605867


In [5]:
# r_formula creates a lme4 type formula from pandas columns
form = r_formula(df, dependent_var='remission', re='(1 | DID)', excluded_cols = ['DID'])
call = "brm(formula, silent = FALSE, data = df, family = bernoulli(), refresh=0, algorithm='meanfield', iter = 1000, chains = 4, cores = 4)"
call = call.replace('formula', form)
call

"brm(remission ~ tumorsize + co2 + pain + wound + mobility + ntumors + nmorphine + lungcapacity + Age + Married + FamilyHx + SmokingHx + Sex + CancerStage + LengthofStay + WBC + RBC + BMI + IL6 + CRP + Experience + School + Lawsuits + HID + Medicaid + (1 | DID), silent = FALSE, data = df, family = bernoulli(), refresh=0, algorithm='meanfield', iter = 1000, chains = 4, cores = 4)"

In [6]:
excall = "brm(remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience + (1 | DID), silent = TRUE, data = df, family = bernoulli(), refresh=0, algorithm='sampling', iter = 1000, chains = 4, cores = 4)"
excall


"brm(remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience + (1 | DID), silent = TRUE, data = df, family = bernoulli(), refresh=0, algorithm='sampling', iter = 1000, chains = 4, cores = 4)"

In [7]:
Xcols = ['IL6', 'CRP', 'CancerStage', 'LengthofStay', 'Experience', 'DID']

In [8]:
ml = skGLMM(r_call = excall)
ml.fit(df[Xcols], df['remission'])

Starting Fit.
 Family: bernoulli 
  Links: mu = logit 
Formula: remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience + (1 | DID) 
   Data: df (Number of observations: 8525) 
Samples: 4 chains, each with iter = 1000; warmup = 500; thin = 1;
         total post-warmup samples = 2000

Group-Level Effects: 
~DID (Number of levels: 407) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     1.99      0.10     1.79     2.19 1.01      379      719

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept       -1.48      0.11    -1.70    -1.26 1.02      210      376
IL6             -0.16      0.03    -0.22    -0.09 1.00     2728     1597
CRP             -0.06      0.03    -0.12    -0.00 1.00     2674     1470
CancerStage     -0.29      0.03    -0.35    -0.22 1.00     2491     1447
LengthofStay    -0.31      0.03    -0.37    -0.25 1.00     2029     1520
Experience       0.49      0.11     0.28

/opt/conda/envs/python2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Compiling the C++ model

  warnings.warn(x, RRuntimeWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Start sampling

  warnings.warn(x, RRuntimeWarning)


In [9]:
ml.summary()

 Family: bernoulli 
  Links: mu = logit 
Formula: remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience + (1 | DID) 
   Data: df (Number of observations: 8525) 
Samples: 4 chains, each with iter = 1000; warmup = 500; thin = 1;
         total post-warmup samples = 2000

Group-Level Effects: 
~DID (Number of levels: 407) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     1.99      0.10     1.79     2.19 1.01      379      719

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept       -1.48      0.11    -1.70    -1.26 1.02      210      376
IL6             -0.16      0.03    -0.22    -0.09 1.00     2728     1597
CRP             -0.06      0.03    -0.12    -0.00 1.00     2674     1470
CancerStage     -0.29      0.03    -0.35    -0.22 1.00     2491     1447
LengthofStay    -0.31      0.03    -0.37    -0.25 1.00     2029     1520
Experience       0.49      0.11     0.28     0.69 1.02

In [10]:
phat = ml.predict(df[Xcols])

/opt/conda/envs/python2/lib/python2.7/site-packages/pyGLMM/_models.py:162: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  X_out = self.x_scaler.transform(X)


In [11]:
# supports sampling from posterior for BRMS and LME4
phatdraws = ml.predict(df[Xcols], n_draws=1000, parallel=True)
phatdraws.shape

  |=======================================================| 100%, Elapsed 00:02


(8525, 1000)

In [12]:
# lme4 example
mlglmer = skGLMM(r_call = "glmer(remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience + (1 | DID), data = df, verbose=1, family = 'binomial', control = glmerControl(optimizer = 'bobyqa'), nAGQ = 10)")
mlglmer.fit(df[Xcols], df['remission'])


Starting Fit.
start par. =  1 fn =  7818.875 
At return
eval:  26 fn:      7620.7853 par:  1.92740
start par. =  1.927397 -1.319264 -0.1477915 -0.06047357 -0.2744597 -0.2988789 0.431074 fn =  7607.932 
At return
eval: 359 fn:      7605.5629 par:  1.96542 -1.47156 -0.153831 -0.0632018 -0.284760 -0.310825 0.482311
Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience +  
    (1 | DID)
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
  7619.6   7668.9  -3802.8   7605.6     8518 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5498 -0.4591 -0.2146  0.4320  4.5955 

Random effects:
 Groups Name        Variance Std.Dev.
 DID    (Intercept) 3.863    1.965   
Number of obs: 8525, groups:  DID, 407

Fixed effects:
             Estimate Std. Error z value Pr(>|z|) 

In [13]:
# and other types of R models
mlglm = skGLMM(r_call = "glm(remission ~ IL6 + CRP + CancerStage + LengthofStay + Experience, data = df, family = 'binomial')")
mlglm.fit(df[Xcols], df['remission'])


Starting Fit.

Call:
glm(formula = remission ~ IL6 + CRP + CancerStage + LengthofStay + 
    Experience, family = "binomial", data = df)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.444  -0.865  -0.714   1.315   2.242  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.91233    0.02470 -36.937  < 2e-16 ***
IL6          -0.10848    0.02515  -4.314 1.61e-05 ***
CRP          -0.05265    0.02465  -2.136   0.0326 *  
CancerStage  -0.17560    0.02567  -6.842 7.82e-12 ***
LengthofStay -0.22023    0.02505  -8.792  < 2e-16 ***
Experience    0.34137    0.02470  13.821  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 10352.6  on 8524  degrees of freedom
Residual deviance:  9983.1  on 8519  degrees of freedom
AIC: 9995.1

Number of Fisher Scoring iterations: 4

R Fit Done. It took 0 seconds


In [14]:
# sklearn support
from sklearn.model_selection import cross_val_score
cross_val_score(mlglm, X=df[Xcols], y=df['remission'], n_jobs=1, cv=5, scoring='neg_mean_squared_error')

Starting Fit.

Call:
glm(formula = remission ~ IL6 + CRP + CancerStage + LengthofStay + 
    Experience, family = "binomial", data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5119  -0.8921  -0.7276   1.3041   2.2263  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.82743    0.02716 -30.463  < 2e-16 ***
IL6          -0.09656    0.02759  -3.500 0.000465 ***
CRP          -0.06324    0.02719  -2.326 0.020029 *  
CancerStage  -0.18442    0.02822  -6.534 6.38e-11 ***
LengthofStay -0.22370    0.02758  -8.110 5.07e-16 ***
Experience    0.35371    0.02734  12.939  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 8480.7  on 6819  degrees of freedom
Residual deviance: 8160.3  on 6814  degrees of freedom
AIC: 8172.3

Number of Fisher Scoring iterations: 4

R Fit Done. It took 0 seconds
Starting Fit.

Call:
glm(formula = re

array([-0.1773876 , -0.20846035, -0.19201361, -0.22890919, -0.19891436])